# Monasca Bootcamp
## Hands on Lab

https://github.com/witekest/monasca-bootcamp

---
### [Roland Hochmuth](https://www.linkedin.com/in/rolandhochmuth)

company: Hewlett Packard Enterprise

email: <roland.hochmuth@hpe.com>

### Witek Bedyk

company: Fujitsu EST

email: <witold.bedyk@est.fujitsu.com>

# The Monasca team attending the Boston Summit

* Stefano Canepa (Hewlett Packard Enterprise)

* James Gu (SUSE)

* Dan ? (SUSE)

* Haruki Yamanashi (Fujitsu Limited)

* Cristiano Bellucci (Fujitsu EST)

* and more

# Running the iPython notebook
---
This is an iPython/Jupyter notebook that you can run on your own. You will require the following installed on your system:

* iPython

    * See, https://ipython.org/install.html

* An OpenStack DevStack install with the Monasca DevStack plugin installed.

    * See, https://github.com/openstack/monasca-log-api/tree/master/devstack

# Running notebook on your worshop instance
---
* Set up the SSH tunnel to your instance
    
    * `ssh -i monasca_workshop.pem -NfL localhost:8888:localhost:8889 ubuntu@<your_instance_ip>`
    
* SSH to your instance and start the notebook

    * `ssh -i monasca_workshop.pem ubuntu@<your_instance_ip>`
    
    * `/opt/jupyter/bin/jupyter notebook --no-browser --port=8889 --notebook-dir monasca-bootcamp/`
    
* Copy the URL and open it in your local browser. Remember to set the port number to `8888`.

* Open `Monasca Bootcamp.ipynb` notebook.

## For Windows users

# Agenda
---
* Architecture and Overview
* API, CLI (python-monascaclient) and client hands-on
* Agent hands-on
* Horizon, Grafana, Kibana hands-on

# Architecture
---
![Monasca Architecture](architecture.png)

# Logging
---
![Logging Architecture](Logging_Architecture.png)

# Start the services

In [1]:
!~/scripts/start_services.sh
!~/scripts/list_services.sh

Starting mysql
Starting rabbitmq-server
Starting apache2
Starting zookeeper
Starting storm-nimbus
Starting storm-supervisor
Starting kafka
Starting influxdb
Starting elasticsearch
Starting memcached
Starting monasca-thresh
Starting monasca-notification
Starting monasca-persister
Starting monasca-log-transformer
Starting monasca-log-persister
Starting monasca-log-metrics
Starting monasca-api
Starting monasca-log-api
Starting monasca-agent
Starting monasca-log-agent
Starting grafana-server
Starting kibana
Service mysql is active.
Service rabbitmq-server is active.
Service apache2 is active.
Service zookeeper is active.
Service storm-nimbus is active.
Service storm-supervisor is active.
Service kafka is active.
Service influxdb is active.
Service elasticsearch is active.
Service memcached is active.
Service monasca-thresh is active.
Service monasca-notification is active.
Service monasca-persister is active.
Service monasca-log-transformer is active.
Service monasca-log-persister is activ

# Import libraries and initialization
---
Let's first import some libraries used in the rest of the notebook.

In [3]:
import datetime
import json
import time

# Import the Monasca and Keystone clients
from monascaclient import client
from monascaclient import ksclient

# Initialize the Keystone and Monasca Client
---
We'll be connecting to the DevStack VM and using the `mini-mon` project and username.

In [4]:
KEYSTONE_URL = 'http://<your_instance_ip>:35357/v3'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'

In [12]:
# Authenticate to Keystone
keystone_client = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD,
                                   project_name=PROJECT_NAME)

# Create the Monasca client
monasca_client = client.Client('2_0', keystone_client.monasca_url, token=keystone_client.token)

# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME

env: OS_PROJECT_NAME=mini-mon
env: OS_PASSWORD=password
env: OS_AUTH_URL=http://144.217.244.18:35357/v3
env: OS_USERNAME=mini-mon


# Using the API
---

The [Monasca API](https://github.com/openstack/monasca-api/blob/master/docs/monasca-api-spec.md) has the following resources:

* Versions

* Metrics

* Metrics Measurements

* Metrics Statistics

* Metrics Names

* Notification Methods

* Alarm Definitions

* Alarms

* Alarms Count

* Alarms State History

Please consult the spec for the full details.

# Metrics
---
* GET, POST /v2.0/metrics

* name (string(255), required) - The name of the metric. Naming conventions for metric names:

    * lowercase, `.` to delimit groups, `_` (snake case) to delimit words, with a unit of measurement often as the suffix.
    
    * e.g. cpu.user_perc, kafka.consumer_lag

* dimensions ({string(255): string(255)}, optional) - A dictionary consisting of (key, value) pairs used to uniquely identify a metric and slice and dice on.

* timestamp (string, required) - The timestamp in milliseconds from the Epoch.

* value (float, required) - Value of the metric.

* value_meta ({string(255): string(2048)}, optional) - A dictionary consisting of (key, value) pairs used to describe the metric.

    * Examples: status_code, msg

* tenant_id: Tenant ID to create metrics on behalf of.

    * This parameter can be used to submit metrics from one tenant, to another.
    * Requires the delegate role.

# Dimensions
---

* A dictionary of (key, value) pairs that are used to uniquely identify a metric.

* Used to slice and dice metrics when querying.

* Examples: hostname, service, component, region zone, resource_id, ...

* Dimensions can be anything you want, but naming conventions should be adopted for consistency.

* Examples of dimension keys are the following: hostname, region, zone, service, component, process, ...

# Example: Metrics request body
---
In this example, we are reporting the status of a HTTP check as a binary value, 0 or 1, for a specific host. The API is down, and the status code and msg returned are returned as meta data.
```
{
	name: http_status,
	dimensions: {
		url: http://service.domain.com:80,
		region: uswest,
		zone: 1,
		service: compute
	}
	timestamp: 1461600900000, /* milliseconds */
	value: 1.0,
	value_meta: {
		status_code: 500,
		msg: Internal server error
	}
}
```

# Help: Create metrics
---

In [6]:
!monasca help metric-create

usage: monasca metric-create [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                             [--value-meta <KEY1=VALUE1,KEY2=VALUE2...>]
                             [--time <UNIX_TIMESTAMP>]
                             [--project-id <CROSS_PROJECT_ID>]
                             <METRIC_NAME> <METRIC_VALUE>

Create metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to create.
  <METRIC_VALUE>        Metric value.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to create a metric dimension. This
                        can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --value-meta <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair for extra information about a value.
       

# Example: Create a metric
Let's create a metric with the metric `openstack.handson_status` and some dimensions 

In [7]:
!monasca metric-create --dimensions region=useast,state=MA,city=Boston,session=monasca openstack.handson_status 1.0

Successfully created metric


# List metrics
Lists all the unique metrics in the system

* GET /v2.0/metrics

* A unique metric is identitifed by it's name and dimensions.

* Starttime, endtime, offset and limit paramaters are available.

* Note, if you are monitoring OpenStack resources, such as VMs, based on the amount of churn (VMs being created/destroyed) in the system and your retention policy, it is recommended to specify the starttime, to limit the amount of metrics returned, such that queries return in a reasonable amount of time.

# Help: List Metrics
---


In [8]:
!monasca help metric-list

usage: monasca metric-list [--name <METRIC_NAME>]
                           [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                           [--starttime <UTC_START_TIME>]
                           [--endtime <UTC_END_TIME>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>] [--tenant-id <TENANT_ID>]

List metrics for this tenant.

Optional arguments:
  --name <METRIC_NAME>  Name of the metric to list.
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00

# Example: List metrics

In [9]:
!monasca metric-list --limit 10

+-------------------+-------------------------------------------+
| name              | dimensions                                |
+-------------------+-------------------------------------------+
| cpu.frequency_mhz | hostname: test-new-instance.localdomain   |
|                   | service: monitoring                       |
| cpu.frequency_mhz | hostname: test-with-user-data.localdomain |
|                   | service: monitoring                       |
| cpu.frequency_mhz | hostname: test1                           |
|                   | service: monitoring                       |
| cpu.idle_perc     | hostname: test-new-instance.localdomain   |
|                   | service: monitoring                       |
| cpu.idle_perc     | hostname: test-with-user-data.localdomain |
|                   | service: monitoring                       |
| cpu.idle_perc     | hostname: test1                           |
|                   | service: monitoring                       |
| cpu.idle

# Exercise 1: List metrics and filter on name, dimensions
---
Extend the following command to filter the list and find the previously generated metric.

In [10]:
!monasca metric-list # put your options here

+--------------------------+------------------+
| name                     | dimensions       |
+--------------------------+------------------+
| openstack.handson_status | city: Boston     |
|                          | region: useast   |
|                          | session: monasca |
|                          | state: MA        |
+--------------------------+------------------+


# Example: Create a function to get metrics using the Monasca client
---
Now let's try to create a function which returns the list of metrics using the Monasca Python client.
We will use the previously instantiated 'monasca_client' object and its 'metrics.list' method.
All the CLI options can be used as named arguments.


In [13]:
def get_metrics(names=[None], dimensions={}, limit=10):
    metrics = []
    for name in names:        
        # Invoke the Monasca client
        metrics = metrics + monasca_client.metrics.list(name=name, dimensions=dimensions, limit=limit)
    return metrics

metrics = get_metrics(['openstack.handson_status'])
print json.dumps(metrics, indent=4)

[
    {
        "dimensions": {
            "city": "Boston", 
            "region": "useast", 
            "session": "monasca", 
            "state": "MA"
        }, 
        "name": "openstack.handson_status", 
        "id": "0"
    }
]


# Metric names
* List all the metric names stored in the service

* GET /v2.0/metrics/names

* Query parameters: tenant_id, offset, limit and dimensions

* Very useful for user-interfaces that need to display the names of all the metrics in the system.

    * Querying all metrics and determining all the distinct metric names is too costly.

# Help: List metric names

In [14]:
!monasca help metric-name-list

usage: monasca metric-name-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>]
                                [--tenant-id <TENANT_ID>]

List names of metrics.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --offset <OFFSET LOCATION>
                        The offset used to paginate the return data.
  --limit <RETURN LIMIT>
                        The amount of data to be returned up to the API
                        maximum limit.
  --tenant-id <TENANT_ID>
                        Retrieve data for t

# Example: List metric names

In [15]:
!monasca metric-name-list --limit 20

+----------------------+
| Name                 |
+----------------------+
| cpu.frequency_mhz    |
| cpu.idle_perc        |
| cpu.idle_time        |
| cpu.percent          |
| cpu.stolen_perc      |
| cpu.system_perc      |
| cpu.system_time      |
| cpu.user_perc        |
| cpu.user_time        |
| cpu.wait_perc        |
| cpu.wait_time        |
| disk.inode_used_perc |
| disk.space_used_perc |
| host_alive_status    |
| io.read_kbytes_sec   |
| io.read_req_sec      |
| io.read_time_sec     |
| io.write_kbytes_sec  |
| io.write_req_sec     |
| io.write_time_sec    |
+----------------------+


# Metrics Measurements
---
Operations for returning measurements.

* GET /v2.0/metrics/measurements

* Name and dimensions supplied as query parameters

* Startime, endtime, offset, limit and group_by supplied.

* Note:

    * If group_by is not specified, metrics must be fully qualified with name and dimensions.

# Help: List Measurements
---

In [16]:
!monasca help measurement-list

usage: monasca measurement-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--endtime <UTC_END_TIME>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>] [--merge_metrics]
                                [--group_by <KEY1,KEY2,...>]
                                [--tenant-id <TENANT_ID>]
                                <METRIC_NAME> <UTC_START_TIME>

List measurements for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to list measurements.
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        paramet

# Excercise 2: List measurement
---
Get the list of 5 last measurements with the metric name 'monasca.thread_count' for service 'monitoring' within the last 5 minutes.

In [19]:
!monasca # put your command here

+----------------------+--------------------------+--------------------------+--------------+------------+
| name                 | dimensions               | timestamp                | value        | value_meta |
+----------------------+--------------------------+--------------------------+--------------+------------+
| monasca.thread_count | hostname: test1          | 2017-05-03T10:47:38.695Z |        9.000 |            |
|                      | component: monasca-agent | 2017-05-03T10:47:53.697Z |        9.000 |            |
|                      | service: monitoring      | 2017-05-03T10:48:08.690Z |        9.000 |            |
|                      |                          | 2017-05-03T10:48:23.813Z |        9.000 |            |
|                      |                          | 2017-05-03T10:48:38.720Z |        9.000 |            |
+----------------------+--------------------------+--------------------------+--------------+------------+


# Exercise 3: List measurements by dimension
---
Get the list of measurements with the metric name 'disk.space_used_perc' within last 2 minutes. Please compare with the metric-list and try out using 'group_by' option.


In [22]:
 # put your command here

+----------------------+---------------------------------------+--------------------------+--------------+------------+
| name                 | dimensions                            | timestamp                | value        | value_meta |
+----------------------+---------------------------------------+--------------------------+--------------+------------+
| disk.space_used_perc | mount_point: /                        | 2017-05-03T10:54:38.562Z |       28.000 |            |
|                      |                                       | 2017-05-03T10:54:53.690Z |       28.000 |            |
|                      |                                       | 2017-05-03T10:55:08.569Z |       28.000 |            |
|                      |                                       | 2017-05-03T10:55:23.593Z |       28.000 |            |
|                      |                                       | 2017-05-03T10:55:38.668Z |       28.000 |            |
|                      |                

# Exercise 4: Create a function to get measurements
---
Let's try to use client's method 'metrics.list_measurements' to get the list of measurements.
Please use the output of the get_metrics() function defined in the previous example as the first argument of the new function. Remember that the metric is defined by its name and the unique set of dimensions (these are attributes of the metric).

In [24]:
def get_measurements(metrics, start_time = None, end_time = None, limit=100):
    measurements = []
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:        
        # Invoke the Monasca client
        # please put your code here
        
        measurements += # put your code here
        
    return measurements

# TEST
metrics = get_metrics(['cpu.user_perc','cpu.system_perc'])
measurements = get_measurements(metrics, limit=2)
assert measurements[0]['name'] == 'cpu.user_perc'
assert len(measurements[0]['measurements']) == 2
assert measurements[1]['name'] == 'cpu.system_perc'

print json.dumps(measurements, indent=4)

[
    {
        "name": "cpu.user_perc", 
        "measurements": [
            [
                "2017-05-03T09:58:23.498Z", 
                21.9, 
                {}
            ], 
            [
                "2017-05-03T09:58:38.485Z", 
                20.4, 
                {}
            ]
        ], 
        "id": "0", 
        "columns": [
            "timestamp", 
            "value", 
            "value_meta"
        ], 
        "dimensions": {
            "hostname": "test1", 
            "service": "monitoring"
        }
    }, 
    {
        "name": "cpu.system_perc", 
        "measurements": [
            [
                "2017-05-03T09:58:23.498Z", 
                12.9, 
                {}
            ], 
            [
                "2017-05-03T09:58:38.485Z", 
                14.5, 
                {}
            ]
        ], 
        "id": "0", 
        "columns": [
            "timestamp", 
            "value", 
            "value_meta"
        ], 
        "dim

# Notification Methods
---
Specify the name, type and address to send a notification to.

* GET, POST, PUT, DELETE, PATCH /v2.0/notification-methods

* Notification methods are associated with actions in alarms and are invoked when alarm state transitions occur.

* Supported notification methods are:

  * Email
  
  * PagerDuty
  
  * Webhooks
  
* Plugins available for:

  * Jira
  
  * HipChat
  
  * Slack

# Delete all existing notifications
Let's first delete all existing notifications to prepare the service for creating new ones.

In [25]:
notifications = monasca_client.notifications.list()

for notification in notifications:
    monasca_client.notifications.delete(
        notification_id = notification['id']
    )

# Help: Create Notification Method
---

In [26]:
!monasca help notification-create

usage: monasca notification-create [--period <PERIOD>]
                                   <NOTIFICATION_NAME> <TYPE> <ADDRESS>

Create notification.

Positional arguments:
  <NOTIFICATION_NAME>  Name of the notification to create.
  <TYPE>               The notification type. Type must be EMAIL, WEBHOOK, or
                       PAGERDUTY.
  <ADDRESS>            A valid EMAIL Address, URL, or SERVICE KEY.

Optional arguments:
  --period <PERIOD>    A period for the notification method. Can only be non
                       zero with webhooks


# Example: Create an email notification method

In [27]:
!monasca notification-create "Email Notification" EMAIL john.doe@domain.com

{
  "name": "Email Notification", 
  "links": [
    {
      "href": "http://144.217.244.18:8070/v2.0/notification-methods/6dc52c12-570b-45fe-acf9-6847f30fffeb", 
      "rel": "self"
    }
  ], 
  "period": 0, 
  "address": "john.doe@domain.com", 
  "type": "EMAIL", 
  "id": "6dc52c12-570b-45fe-acf9-6847f30fffeb"
}


# Help: List Notification Methods
---

In [28]:
!monasca help notification-list

usage: monasca notification-list [--sort-by <SORT BY FIELDS>]
                                 [--offset <OFFSET LOCATION>]
                                 [--limit <RETURN LIMIT>]

List notifications for this tenant.

Optional arguments:
  --sort-by <SORT BY FIELDS>
                        Fields to sort by as a comma separated list. Valid
                        values are id, name, type, address, created_at,
                        updated_at. Fields may be followed by "asc" or "desc",
                        ex "address desc", to set the direction of sorting.
  --offset <OFFSET LOCATION>
                        The offset used to paginate the return data.
  --limit <RETURN LIMIT>
                        The amount of data to be returned up to the API
                        maximum limit.


# Example: List Notification Methods

In [29]:
!monasca notification-list

+--------------------+--------------------------------------+-------+---------------------+--------+
| name               | id                                   | type  | address             | period |
+--------------------+--------------------------------------+-------+---------------------+--------+
| Email Notification | 6dc52c12-570b-45fe-acf9-6847f30fffeb | EMAIL | john.doe@domain.com | 0      |
+--------------------+--------------------------------------+-------+---------------------+--------+


# Alarm Definitions
---
Operations for creating, reading updating and deleting alarm definitions.

* GET, POST /v2.0/alarm-definitions

* GET, PUT, PATCH, DELETE /v2.0/alarm-definitions/{alarm-definition-id}

* Alarm definitions are templates that are used to automatically create alarms based on matching metric names and dimensions

    * The `match-by` paramater is used to match/group metrics together by dimension
    
    * e.g. `--match-by hostname` will create an alarm per unique hostname.

* One alarm definition can result in many alarms

* Simple grammar for creating compound alarm expressions:

   avg(cpu.user_perc{}) > 85 or avg(disk.read_ops{device=vda}, 120) > 1000


* Alarm state (ALARM, OK and UNDETERMINED)

* Actions (notification methods) associated with alarms for state transitions

* User assigned severity (LOW, MEDIUM, HIGH, CRITICAL)



# Delete existing alarm definitions
Let's first delete all existing alarm definitions to prepare the service for creating new ones.

In [30]:
alarm_definitions = monasca_client.alarm_definitions.list()

for definition in alarm_definitions:
    monasca_client.alarm_definitions.delete(alarm_id=definition['id'])    

# Help: Create alarm definition

In [31]:
!monasca help alarm-definition-create

usage: monasca alarm-definition-create [--description <DESCRIPTION>]
                                       [--severity <SEVERITY>]
                                       [--match-by <MATCH_BY_DIMENSION_KEY1,MATCH_BY_DIMENSION_KEY2,...>]
                                       [--alarm-actions <NOTIFICATION-ID>]
                                       [--ok-actions <NOTIFICATION-ID>]
                                       [--undetermined-actions <NOTIFICATION-ID>]
                                       <ALARM_DEFINITION_NAME> <EXPRESSION>

Create an alarm definition.

Positional arguments:
  <ALARM_DEFINITION_NAME>
                        Name of the alarm definition to create.
  <EXPRESSION>          The alarm expression to evaluate. Quoted.

Optional arguments:
  --description <DESCRIPTION>
                        Description of the alarm.
  --severity <SEVERITY>
                        Severity is one of [LOW, MEDIUM, HIGH, CRITICAL].
  --match-by <MATCH_BY_DIMENSION_KEY1,MATCH_BY_DIM

# Example: Create alarm definition
Create an alarm definition that triggers if the avg of CPU user percent is greater than 80% over 3 60 s periods.

In [32]:
!monasca alarm-definition-create simple-alarm "max(cpu.user_perc{}, 60) > 80 times 3"

{
  "alarm_actions": [], 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://144.217.244.18:8070/v2.0/alarm-definitions/9ca25671-2d27-4bb2-9590-e1b4101b268a", 
      "rel": "self"
    }
  ], 
  "match_by": [], 
  "deterministic": false, 
  "name": "simple-alarm", 
  "actions_enabled": "true", 
  "undetermined_actions": [], 
  "expression": "max(cpu.user_perc{}, 60) > 80 times 3", 
  "id": "9ca25671-2d27-4bb2-9590-e1b4101b268a", 
  "severity": "LOW"
}


# Example: Create a compound alarm definition
An alarm definition that triggers if either the CPU user or system percent is greater than 80% over 3 60 second periods.

In [33]:
!monasca alarm-definition-create compound-alarm "avg(cpu.user_perc{}, 60) > 80 times 3 or avg(cpu.system_perc{}, 60) > 80 times 3"

{
  "alarm_actions": [], 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://144.217.244.18:8070/v2.0/alarm-definitions/ab2cce2d-4337-4d46-8dfe-2c0971944dd1", 
      "rel": "self"
    }
  ], 
  "match_by": [], 
  "deterministic": false, 
  "name": "compound-alarm", 
  "actions_enabled": "true", 
  "undetermined_actions": [], 
  "expression": "avg(cpu.user_perc{}, 60) > 80 times 3 or avg(cpu.system_perc{}, 60) > 80 times 3", 
  "id": "ab2cce2d-4337-4d46-8dfe-2c0971944dd1", 
  "severity": "LOW"
}


# Alarm definition match_by

* As metrics are consumed by the Threshold Engine they are filtered and checked if they match the definitions that have been specified, based on the metric name and dimensions.

* If a match is found and it is the first one, a new alarm is created.

* If the match_by parameter is specified, metrics are grouped into alarms that match the specified dimensions.

# Example: Two alarm definitions
1. One with no match_by parameter
2. One with match_by set on hostname

In [34]:
!monasca alarm-definition-create "match name only" "max(test-metric{}) > 80"
!monasca alarm-definition-create --match-by hostname "match by hostname" "max(test-metric{}) > 80"

{
  "alarm_actions": [], 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://144.217.244.18:8070/v2.0/alarm-definitions/6a78e299-2476-44da-9ede-fb2f24288cb8", 
      "rel": "self"
    }
  ], 
  "match_by": [], 
  "deterministic": false, 
  "name": "match name only", 
  "actions_enabled": "true", 
  "undetermined_actions": [], 
  "expression": "max(test-metric{}) > 80", 
  "id": "6a78e299-2476-44da-9ede-fb2f24288cb8", 
  "severity": "LOW"
}
{
  "alarm_actions": [], 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://144.217.244.18:8070/v2.0/alarm-definitions/3875b0ef-d445-4436-9938-a8f9439462fa", 
      "rel": "self"
    }
  ], 
  "match_by": [
    "hostname"
  ], 
  "deterministic": false, 
  "name": "match by hostname", 
  "actions_enabled": "true", 
  "undetermined_actions": [], 
  "expression": "max(test-metric{}) > 80", 
  "id": "3875b0ef-d445-4436-9938-a8f9439462fa", 
  "severity": "LOW"
}


# Example: Send two metrics
1. One with a name of test-metric and a hostname set to foo
2. One with a name of test-metric and a hostname set to bar

In [35]:
!monasca metric-create --dimensions hostname=foo test-metric 0.0
!monasca metric-create --dimensions hostname=bar test-metric 0.0

Successfully created metric
Successfully created metric


# Quiz: How many alarms were created?

In [36]:
!monasca alarm-list --metric-name "test-metric"

+--------------------------------------+--------------------------------------+-----------------------+-------------+-------------------+----------+--------------+-----------------+------+-------------------------+----------------------+----------------------+
| id                                   | alarm_definition_id                  | alarm_definition_name | metric_name | metric_dimensions | severity | state        | lifecycle_state | link | state_updated_timestamp | updated_timestamp    | created_timestamp    |
+--------------------------------------+--------------------------------------+-----------------------+-------------+-------------------+----------+--------------+-----------------+------+-------------------------+----------------------+----------------------+
| 32878f49-f83b-41f6-a81c-c65c45a1b45e | 3875b0ef-d445-4436-9938-a8f9439462fa | match by hostname     | test-metric | hostname: foo     | LOW      | UNDETERMINED | None            | None | 2017-05-03T11:00:36Z    | 20

Three alarms were created:

1. There is one alarm with name "match name only". It has two metrics that are associated with it.

2. There are two alarms with name "match by hostname". Each alarm has one metric that is associated with it, based on the hostnames of `foo` and `bar`.

# UNDETERMINED alarm state
* There are three states of an alarm, `OK`, `ALARM` and `UNDETERMINED`.

* The `UNDETERMINED` alarm state occurs when metrics are no longer being received by an alarm.

* For example, a service is disabled, a system goes down, the network experiences a disruption.

* You can define the alarm as `deterministic`. The only allowed states are then: `OK` and `ALARM`.

# TODO Example: Create an alarm definition with an alarm action.
## Note to myself: update these cells

In [38]:
!monasca alarm-definition-show 6a78e299-2476-44da-9ede-fb2f24288cb8

+----------------------+------------------------------------------------------------------------------------------------------+
| Property             | Value                                                                                                |
+----------------------+------------------------------------------------------------------------------------------------------+
| actions_enabled      | true                                                                                                 |
| alarm_actions        | []                                                                                                   |
| description          | ""                                                                                                   |
| deterministic        | False                                                                                                |
| expression           | "max(test-metric{}) > 80"                                                      

In [ ]:
# Get the ID of the pagerduty notification method
notifications = monasca_client.notifications.list(sort_by='TYPE')
pagerduty_notification_id = None
for notification in notifications:
    if notification['type'] == 'PAGERDUTY':
        pagerduty_notification = notification   
        pagerduty_notification_id = pagerduty_notification['id']

# Create the alarm        
monasca_client.alarm_definitions.create(name='bootcamp-test',
                                        expression='max(cpu.user_perc{}) > 0',
                                        alarm_actions=pagerduty_notification_id)

# Alarms
---
Alarms are created when incoming metrics match alarm definitions


* GET /v2.0/alarms
* GET, PUT, PATCH, DELETE /v2.0/alarms/{alarm-id}

* Query Parameters

    * alarm_definition_id (string, optional) - Alarm definition ID to filter by.

    * metric_name (string(255), optional) - Name of metric to filter by.

    * metric_dimensions ({string(255): string(255)}, optional) - Dimensions of metrics to filter by specified as a comma separated array of (key, value) pairs as `key1:value1,key1:value1, ...`

    * state (string, optional) - State of alarm to filter by, either `OK`, `ALARM` or `UNDETERMINED`.

    * state_updated_start_time (string, optional) - The start time in ISO 8601 combined date and time format in UTC.
    
    * sort-by: Fields to sort by
    
    * offset, limit


# Help: Alarms list

In [39]:
!monasca help alarm-list

usage: monasca alarm-list [--alarm-definition-id <ALARM_DEFINITION_ID>]
                          [--metric-name <METRIC_NAME>]
                          [--metric-dimensions <KEY1=VALUE1,KEY2,KEY3=VALUE2...>]
                          [--state <ALARM_STATE>] [--severity <SEVERITY>]
                          [--state-updated-start-time <UTC_STATE_UPDATED_START>]
                          [--lifecycle-state <LIFECYCLE_STATE>]
                          [--link <LINK>] [--sort-by <SORT BY FIELDS>]
                          [--offset <OFFSET LOCATION>]
                          [--limit <RETURN LIMIT>]

List alarms for this tenant.

Optional arguments:
  --alarm-definition-id <ALARM_DEFINITION_ID>
                        The ID of the alarm definition.
  --metric-name <METRIC_NAME>
                        Name of the metric.
  --metric-dimensions <KEY1=VALUE1,KEY2,KEY3=VALUE2...>
                        key value pair used to specify a metric dimension or
                        just key t

# Example: Alarm list

In [40]:
!monasca alarm-list --sort-by severity

+--------------------------------------+--------------------------------------+-----------------------+-----------------+---------------------+----------+--------------+-----------------+------+-------------------------+----------------------+----------------------+
| id                                   | alarm_definition_id                  | alarm_definition_name | metric_name     | metric_dimensions   | severity | state        | lifecycle_state | link | state_updated_timestamp | updated_timestamp    | created_timestamp    |
+--------------------------------------+--------------------------------------+-----------------------+-----------------+---------------------+----------+--------------+-----------------+------+-------------------------+----------------------+----------------------+
| 1d22bb16-cca1-4569-ba2a-03c3fa46a296 | ab2cce2d-4337-4d46-8dfe-2c0971944dd1 | compound-alarm        | cpu.user_perc   | hostname: test1     | LOW      | OK           | None            | None | 2017

# Alarm Counts
---
Provides the ability to query counts of alarms using a number of filter and group-by query parameters.

* GET /v2.0/alarms/counts

* Primarily used in summary/overview dashboards to show the number of alarms in the OK, ALARM and UNDETERMINED state.

* Queries are processed in-database, not client-side.

* A number of filter and group-by fields are supported.

# Help: Alarm counts

In [41]:
!monasca help alarm-count

usage: monasca alarm-count [--alarm-definition-id <ALARM_DEFINITION_ID>]
                           [--metric-name <METRIC_NAME>]
                           [--metric-dimensions <KEY1=VALUE1,KEY2,KEY3=VALUE2...>]
                           [--state <ALARM_STATE>] [--severity <SEVERITY>]
                           [--state-updated-start-time <UTC_STATE_UPDATED_START>]
                           [--lifecycle-state <LIFECYCLE_STATE>]
                           [--link <LINK>] [--group-by <GROUP_BY>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>]

Count alarms.

Optional arguments:
  --alarm-definition-id <ALARM_DEFINITION_ID>
                        The ID of the alarm definition.
  --metric-name <METRIC_NAME>
                        Name of the metric.
  --metric-dimensions <KEY1=VALUE1,KEY2,KEY3=VALUE2...>
                        key value pair used to specify a metric dimension or
                        just key to select al

# Example: Alarm counts

In [42]:
!monasca alarm-count --metric-dimensions service=monitoring --group-by state,dimension_name

+-------+-------+----------------+
| count | state | dimension_name |
+-------+-------+----------------+
| 2     | OK    | hostname       |
| 2     | OK    | service        |
+-------+-------+----------------+


# Metrics Agent
---
* A Python monitoring agent
* Push model
* Agent is installed on the systems that we want to monitor
* Collects metrics by running a set of collection plugins every X amount of seconds 
* Collection plugins are enabled by detection plugins
* Detection plugins generate yaml config files that the collection plugins read from
* The agent has a monasca-setup command line tool that helps configure the agent and run detection plugins

# Agent Metrics and Plugins

* System metrics (cpu, memory, network, filesystem, …)

* Service metrics

* RabbitMQ, MySQL, Kafka, and many others

* Application metrics

    * Built-in statsd daemon

    * Python monasca-statsd library: Adds support for dimensions

* VM system metrics

* Active checks

    * HTTP status checks and response times
    
    * System up/down checks (ping and ssh)
    
* Support for Nagios and checkmk

* Extensible/Pluggable: Additional services can be easily added


# Monasca-setup options

In [43]:
!/opt/monasca-agent/bin/monasca-setup -h

usage: monasca-setup [-h] [-u USERNAME] [-p PASSWORD]
                     [--user_domain_id USER_DOMAIN_ID]
                     [--user_domain_name USER_DOMAIN_NAME]
                     [--keystone_url KEYSTONE_URL]
                     [--project_name PROJECT_NAME]
                     [--project_domain_id PROJECT_DOMAIN_ID]
                     [--project_domain_name PROJECT_DOMAIN_NAME]
                     [--project_id PROJECT_ID] [--monasca_url MONASCA_URL]
                     [--service_type SERVICE_TYPE]
                     [--endpoint_type ENDPOINT_TYPE]
                     [--region_name REGION_NAME] [--system_only]
                     [-d [DETECTION_PLUGINS [DETECTION_PLUGINS ...]]]
                     [--skip_detection_plugins [SKIP_DETECTION_PLUGINS [SKIP_DETECTION_PLUGINS ...]]]
                     [-a DETECTION_ARGS | -json DETECTION_ARGS_JSON]
                     [--check_frequency CHECK_FREQUENCY]
                     [--num_collector_threads NUM_COLLECTOR_TH

# Monasca-setup configuring agent

In [44]:
!sudo cat /usr/local/bin/monasca-reconfigure

#!/bin/sh
'/opt/monasca-agent/bin/monasca-setup' \
    -u 'monasca-agent' \
    -p 'password' \
     -s 'monitoring'  \
    --keystone_url 'http://144.217.244.18:35357/v3' \
    --project_name 'mini-mon' \
     --monasca_url 'http://144.217.244.18:8070/v2.0'  \
     \
     --check_frequency '15'  \
     \
     --log_level 'WARN'  \
    --overwrite \
    --system_only \
    --monasca_statsd_port 8125


# Monasca agent configuration file

In [45]:
!sudo cat /etc/monasca/agent/agent.yaml

Api:
  amplifier: 0
  backlog_send_rate: 1000
  ca_file: null
  endpoint_type: null
  insecure: false
  keystone_url: http://144.217.244.18:35357/v3
  max_buffer_size: 1000
  max_measurement_buffer_size: -1
  password: password
  project_domain_id: null
  project_domain_name: null
  project_id: null
  project_name: mini-mon
  region_name: null
  service_type: null
  url: http://144.217.244.18:8070/v2.0
  user_domain_id: null
  user_domain_name: null
  username: monasca-agent
Logging:
  collector_log_file: /var/log/monasca/agent/collector.log
  forwarder_log_file: /var/log/monasca/agent/forwarder.log
  log_level: WARN
  statsd_log_file: /var/log/monasca/agent/statsd.log
Main:
  check_freq: 15
  collector_restart_interval: 24
  dimensions:
    service: monitoring
  hostname: localhost
  num_collector_threads: 1
  pool_full_max_retries: 4
  sub_collection_warn: 6
Statsd:
  monasca_statsd_port: 8125


# Agent Detection Plugins

* Run after initial configuration is run
* List of avaiable plugins 


# Run Kafka Detection Plugin

In [ ]:
!sudo /opt/monasca-agent/bin/monasca-setup -d kafka

# Example: Detection yaml configuration

In [46]:
!sudo cat /etc/monasca/agent/conf.d/kafka_consumer.yaml

cat: /etc/monasca/agent/conf.d/kafka_consumer.yaml: No such file or directory


# Example: Query the Kafka consumer lag metrics

In [ ]:
!monasca metric-list --name kafka.consumer_lag

# Example: Query the Kafka consumer lag measurements

In [ ]:
!monasca measurement-list kafka.consumer_lag --dimensions consumer_group=1_metrics -10 --limit 10

# Logs Agent
---
* Based on Logstash (alternatively Beaver)

* Uses plugins

    * file for input
    
    * [monasca-log-api](https://github.com/logstash-plugins/logstash-output-monasca_log_api) for output

# Example: Examine the configuration of log agent

In [47]:
!cat /opt/stack/monasca-log-agent/agent.conf

#
# Copyright 2016 FUJITSU LIMITED
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
input {
  file {
    add_field => { "dimensions" => { "service" => "system" }}
    path => "/var/log/syslog"
    tags => ["syslog"]
  }
}

filter {
  if "syslog" in [tags] {
    multiline {
      negate => "true"
      pattern => "^%{SYSLOGTIMESTAMP}"
      what => "previous"
    }
  }
}

# TODO(trebskit) should use own user for log-agent
output {
  monasca_log_api {
    monasca_log_api_url => "http://144.217.244

# Example: Collect logs of metric agent
---
* Edit the configuration of log-agent

    nano /opt/stack/monasca-log-agent/agent.conf
    
* Add the following block to the input plugin configuration

    file {
        add_field => { "dimensions" => { "service" => "monasca" }}
        add_field => { "dimensions" => { "component" => "monasca-agent" }}
        path => "/var/log/monasca/agent/*"
    }
    
* Save the new configuration    

# Test the syntax of new configuration

In [48]:
!/opt/stack/logstash/bin/logstash -f /opt/stack/monasca-log-agent/agent.conf --configtest

Configuration OK


# Restart the service

In [ ]:
!sudo systemctl restart monasca-log-agent

# Horizon, Grafana and Kibana Demo
---
* Log in to the Horizon dashbord 
    
    * URL: `http://<your_instance_ip>`
    
    * User: mini-mon
    
    * Password: password

# Monitoring Dashboard
---
* Explore the panels

    * Overview
    * Alarm Definitions
    * Alarms
    * Notifications

# Grafana 4
---
* Open Grafana from Monitoring Overview

![Open_Grafana](Open_Grafana.png)

# Add Monasca Datasource
---
* Select *Data Sources -> Add data source*
* Configuration:

    * Name: monasca
    * Type: Monasca
    * Default: yes
    * Url: `http://<your_instance_ip>:8070`
    * Auth: Keystone Auth
    
![Add Data Source](Add_DataSource.png)

# Kibana
---
* Open Log Management from Monitoring Overview
* Monasca Kibana plugin enhancements:
    * OpenStack user role based authorization
    * Set default index pattern based on the current project
    * Filter queries to authorized project only
* Explorce the panels:
    * Discover
    * Visualize
    * Dashboard

# Contact the Monasca Team
---
Here are a couple of ways of getting in-touch with us.

* Monasca Weekly Meetings

    * Weekly on Wednesday at 1500 UTC in #openstack-meeting-3 (IRC webclient)
    
* Monasca in IRC

    * #openstack-monasca

# Thank you
![Thank you](austin-thank-you.png)

# Solution ex. 1:

In [ ]:
!monasca metric-list --name openstack.handson_status --dimensions 'city=Boston'

# Solution ex. 2:

In [ ]:
!monasca measurement-list --dimensions 'service=monitoring' --limit 5 monasca.thread_count -5   

# Solution ex. 3:

In [ ]:
!monasca metric-list --name disk.space_used_perc
!monasca measurement-list --group_by mount_point disk.space_used_perc -2

# Solution ex. 4:

In [ ]:
measurements += monasca_client.metrics.list_measurements(
                name=metric['name'],
                dimensions=metric['dimensions'],
                start_time=start_time,
                end_time=end_time,
                limit=limit)